In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [ ]:
user_ratings_df = pd.read_csv("Dataset.csv")
user_ratings_df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [ ]:
movie_metadata = pd.read_csv("Movie_Id_Titles.csv")
movie_metadata.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [ ]:
movie_data = user_ratings_df.merge(movie_metadata, on='item_id')
movie_data.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [ ]:
user_item_matrix = movie_data.pivot(index=['user_id'], columns=['item_id'], values='rating').fillna(0)
user_item_matrix

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
cf_knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
cf_knn_model.fit(user_item_matrix.values.T)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [ ]:
def get_movie_id(movie_name, movie_metadata):
    movie_id = movie_metadata[movie_metadata['title'] == movie_name]['item_id'].iloc[0]
    return movie_id

In [ ]:
def get_knn_recommendations(movie_name, knn_model, user_item_matrix, movie_metadata, num_recommendations):
    movie_id = get_movie_id(movie_name, movie_metadata)

    distances, indices = knn_model.kneighbors(user_item_matrix.loc[:, movie_id].values.reshape(1, -1), n_neighbors=num_recommendations + 1)

    recommendations = []
    for i in range(1, len(indices.flatten())):
        recommended_movie_id = user_item_matrix.columns[indices.flatten()[i]]
        recommended_movie_name = movie_metadata[movie_metadata['item_id'] == recommended_movie_id]['title'].values[0]
        recommendations.append(recommended_movie_name)

    return recommendations

In [ ]:
movie_name = 'Copycat (1995)'
recommendations = get_knn_recommendations(movie_name, cf_knn_model, user_item_matrix, movie_metadata, 5)
print(recommendations)

['Cape Fear (1991)', 'Nightmare on Elm Street, A (1984)', 'Outbreak (1995)', 'Jaws (1975)', 'Natural Born Killers (1994)']
